# Income AQI

#### Data Acquisition
I assume that you get the data something like

In [1]:
import pandas as pd

In [2]:
# df1: income data
df1 = pd.DataFrame({
    'state': ['California', 'Texas', 'New York', 'Florida', 'Illinois'],
    'county': ['Los Angeles', 'Harris', 'Kings', 'Miami-Dade', 'Cook'],
    'income': [75000, 65000, 72000, 58000, 61000]
})

# df2: aqi data
df2 = pd.DataFrame({
    'state': ['California', 'Texas', 'New York', 'Florida', 'Illinois'],
    'county': ['Los Angeles', 'Harris', 'Kings', 'Miami-Dade', 'Cook'],
    'aqi': [90, 80, 60, 55, 70]
})

#### Goal: Create an interactive plot with the U.S. map

You can likely find interactive plots similar to your goal in the [plotly graphing libraries](https://plotly.com/python/)!

- In this example, we use [Choropleth Maps](https://plotly.com/python/choropleth-maps/).
- According to the documentation, we need a [FIPS code](https://en.wikipedia.org/wiki/FIPS_county_code) for each county.
- *But how can we get this?*

I am pretty sure that you are *not the first person in the world* trying to obtain FIPS codes for each county. There are probably existing packages or public resources that can help. Start by searching the [Conda-forge](https://anaconda.org/conda-forge/) website since it's more reliable.

#### Obtaining a FIPS code

In this example, we use [Addfips](https://anaconda.org/conda-forge/addfips). You can check the commands to install this package and documentation of the package in the link. The only thing you require to do before the installation is activating `cosmos` environment. Follow these steps:
1. Open a new terminal (either on VS Code or system terminal)
2. Run 
```zsh
conda activate cosmos
conda install conda-forge::addfips
```

After installation, you can load this package by `import`

In [3]:
import addfips

Now, we want to add `FIPS` column to each dataframe.

In [4]:
af = addfips.AddFIPS()

for df in [df1, df2]:
    df['FIPS'] = df.apply(lambda row: af.get_county_fips(row['county'], row['state']), axis=1)

cols1 = ['FIPS'] + [col for col in df1.columns if col != 'FIPS']
df1 = df1[cols1]

cols2 = ['FIPS'] + [col for col in df2.columns if col != 'FIPS']
df2 = df2[cols2]

In [5]:
df1

,FIPS,state,county,income
0,06037,California,Los Angeles,75000
1,48201,Texas,Harris,65000
2,36047,New York,Kings,72000
3,12086,Florida,Miami-Dade,58000
4,17031,Illinois,Cook,61000


In [6]:
df2

,FIPS,state,county,aqi
0,06037,California,Los Angeles,90
1,48201,Texas,Harris,80
2,36047,New York,Kings,60
3,12086,Florida,Miami-Dade,55
4,17031,Illinois,Cook,70


#### Choropleth Maps

We also need data that links FIPS codes to their geographic shapes—this is where a `geojson` file comes in!

In [7]:
from urllib.request import urlopen
import json

In [8]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

We are now ready to create a plot. You might not be familiar with the necessary commands yet—but don’t worry! 

You can refer to examples in the official documentation or consult an AI assistant. 

Just remember: when using an AI assistant like GPT, always **double-check** that the code if it provides matches your specific needs.

I got this code from ChatGPT and didn't revise anything. Let's see how it works.

In [ ]:
import plotly.graph_objects as go

# Choropleth
fig = go.Figure()

# Income trace
fig.add_trace(go.Choropleth(
    geojson=counties,
    locations=df1['FIPS'],
    z=df1['income'],
    colorscale='Greens',
    zmin=min(df1['income'].min(), df2['aqi'].min()),
    zmax=max(df1['income'].max(), df2['aqi'].max()),
    colorbar_title="Income",
    marker_line_width=0,
    visible=True,
    name="Income"
))

# AQI trace
fig.add_trace(go.Choropleth(
    geojson=counties,
    locations=df2['FIPS'],
    z=df2['aqi'],
    colorscale='Reds',
    zmin=min(df1['income'].min(), df2['aqi'].min()),
    zmax=max(df1['income'].max(), df2['aqi'].max()),
    colorbar_title="AQI",
    marker_line_width=0,
    visible=False,
    name="AQI"
))

# Layout and Dropdown menu
fig.update_layout(
    title_text="County-Level Income and Air Quality (AQI)",
    autosize=False,
    geo_scope='usa',
    updatemenus=[dict(
        active=0,
        buttons=list([
            dict(label="Income",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"coloraxis": {"colorbar": {"title": "Income"}},
                        "title": "County-Level Income"}]),
            dict(label="AQI",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"coloraxis": {"colorbar": {"title": "AQI"}},
                        "title": "County-Level Air Quality Index"}])
        ]),
        direction="down",
        showactive=True
    )]
)

fig.show()

*Any issues here?*

In [ ]:
# Choropleth
fig = go.Figure()

# Income trace
fig.add_trace(go.Choropleth(
    geojson=counties,
    locations=df1['FIPS'],
    z=df1['income'],
    colorscale='Greens',
    zmin=df1['income'].min() * 0.8, # Revised
    zmax=df1['income'].max(), # Revised
    colorbar_title="Income",
    marker_line_width=0,
    visible=True,
    name="Income"
))

# AQI trace
fig.add_trace(go.Choropleth(
    geojson=counties,
    locations=df2['FIPS'],
    z=df2['aqi'],
    colorscale='Reds',
    zmin=0, # Revised
    zmax=df2['aqi'].max(), # Revised
    colorbar_title="AQI",
    marker_line_width=0,
    visible=False,
    name="AQI"
))

# Layout and Dropdown menu
fig.update_layout(
    title_text="County-Level Income and Air Quality (AQI)",
    autosize=False,
    geo_scope='usa',
    updatemenus=[dict(
        active=0,
        buttons=list([
            dict(label="Income",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"coloraxis": {"colorbar": {"title": "Income"}},
                        "title": "County-Level Income"}]),
            dict(label="AQI",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"coloraxis": {"colorbar": {"title": "AQI"}},
                        "title": "County-Level Air Quality Index"}])
        ]),
        direction="down",
        showactive=True
    )]
)

fig.show()

#### Export to HTML

If you like this plot, you can export it to HTML for embedding in a webpage by the command `fig.write_html("FILENAME")`.

In [11]:
fig.write_html("income_aqi.html")

HTML itself is a webpage, and most browsers support it. If you open the file, then you can see the plot in your web browser.

#### Embed to your website

Since HTML is a webpage, if you want to include it to your website, what you want to do is include a webpage to a current webpage. You can use `iframe`! Follow these step:

1. Go to the website repository on Github.
2. Create `static/plotly/.gitkeep` file and upload your html file (e.g. `income_aqi.html` in this example) to `static/plotly`.
3. Go to your markdown file where you want to put the plot. In this example, I'll put it on `content/project/_index.md`.
4. Type `<iframe src="/plotly/YOUR_FIGURE_NAME.html" width="100%" height="400px" style="border:none;"></iframe>`. Some explanations for attributes:
    - `src`: the content to embed into this iframe
    - `width`: width of the frame. "100%" works for most cases.
    - `height`: height of the frame.
    - `style`: style of the frame. By setting `"border:none"`, it does not have any border for this frame.
5. The website will be ready in few minutes.

Few notes:
- The plot may not display well on mobile devices.
- If the HTML file is large, you might experience some lag.